In [ ]:

# 1. Setup and Installation

print("--- 1. Installing Required Libraries ---")
!pip install -q -U transformers peft accelerate bitsandbytes trl datasets scipy

import torch
import os
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import pandas as pd

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


--- 1. Installing Required Libraries ---
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 15.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pylibcudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.6.0 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
Using device: cuda


In [ ]:

# 2. Configuration Variables
model_id = "microsoft/Phi-3-mini-4k-instruct"
output_dir = "./phi3-bfsi-finetuned"
lora_rank = 16
lora_alpha = 32
lora_dropout = 0.05
learning_rate = 2e-4
batch_size = 4
num_train_epochs = 2
max_seq_length = 512 # Limiting sequence length for Colab memory
gradient_accumulation_steps = 2
warmup_ratio = 0.03
logging_steps = 10


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving fraud_chat_data (1).jsonl to fraud_chat_data (1).jsonl


In [ ]:
# 3. Data Preparation (Synthetic BFSI Dataset)
print("\n--- 2. Preparing Synthetic BFSI Dataset ---")

bfsi_data = [
  {
    "prompt": "What is the primary difference between a Mutual Fund and an ETF?",
    "answer": "A Mutual Fund is generally priced once per day (Net Asset Value or NAV) after the market closes, whereas an ETF trades throughout the day on an exchange like a stock, offering real-time pricing and liquidity."
  },
  {
    "prompt": "Explain the concept of 'Basel III' in simple terms.",
    "answer": "Basel III is a global, voluntary regulatory framework on bank capital adequacy, stress testing, and market liquidity risk. Its goal is to make banks stronger and safer."
  },
  {
    "prompt": "Define 'systemic risk' in the context of banking.",
    "answer": "Systemic risk refers to the risk of collapse of an entire financial system or market, not just a single institution. It happens when problems at one bank spread to others."
  },
  {
    "prompt": "What does GDP stand for and what does it measure?",
    "answer": "GDP stands for Gross Domestic Product. It measures the total value of all goods and services produced within a country during a specific time period."
  },
  {
    "prompt": "What is inflation?",
    "answer": "Inflation is the rate at which the general level of prices for goods and services rises, decreasing the purchasing power of money over time."
  },
  {
    "prompt": "What is deflation?",
    "answer": "Deflation is the decrease in the general price level of goods and services, usually caused by reduced spending and demand in the economy."
  },
  {
    "prompt": "What is the difference between fiscal policy and monetary policy?",
    "answer": "Fiscal policy involves government spending and taxation decisions, while monetary policy is managed by the central bank and controls money supply and interest rates."
  },
  {
    "prompt": "What is a central bank?",
    "answer": "A central bank is a national institution that manages the country’s money supply, interest rates, and financial stability. Examples include the Federal Reserve and the European Central Bank."
  },
  {
    "prompt": "What is a bond?",
    "answer": "A bond is a fixed-income investment where an investor lends money to an entity (like a government or company) in exchange for periodic interest payments and the return of principal at maturity."
  },
  {
    "prompt": "What is a stock?",
    "answer": "A stock represents ownership in a company. When you buy a stock, you own a small part of that company and may receive dividends."
  },
  {
    "prompt": "What is diversification in investing?",
    "answer": "Diversification is an investment strategy that involves spreading investments across different assets to reduce overall risk."
  },
  {
    "prompt": "What does ROI stand for?",
    "answer": "ROI stands for Return on Investment. It measures the gain or loss from an investment relative to its cost."
  },
  {
    "prompt": "What is an IPO?",
    "answer": "IPO stands for Initial Public Offering. It’s when a company sells its shares to the public for the first time."
  },
  {
    "prompt": "What is a recession?",
    "answer": "A recession is a significant decline in economic activity across the economy lasting more than a few months, often marked by falling GDP and rising unemployment."
  },
  {
    "prompt": "What is a bull market?",
    "answer": "A bull market is a period when stock prices are rising or expected to rise, reflecting investor confidence."
  },
  {
    "prompt": "What is a bear market?",
    "answer": "A bear market is a period when stock prices are falling or expected to fall, often due to pessimism or weak economic performance."
  },
  {
    "prompt": "What is liquidity?",
    "answer": "Liquidity refers to how easily an asset can be converted into cash without significantly affecting its price."
  },
  {
    "prompt": "What is market capitalization?",
    "answer": "Market capitalization, or market cap, is the total market value of a company’s outstanding shares, calculated by multiplying share price by total shares."
  },
  {
    "prompt": "What is a derivative?",
    "answer": "A derivative is a financial contract whose value is based on an underlying asset, index, or rate, such as futures or options."
  },
  {
    "prompt": "What does 'hedging' mean in finance?",
    "answer": "Hedging is a risk management strategy used to offset potential losses in one investment by taking an opposite position in a related asset."
  },
  {
    "prompt": "What is credit risk?",
    "answer": "Credit risk is the risk that a borrower will fail to repay a loan or meet contractual obligations."
  },
  {
    "prompt": "What is interest rate risk?",
    "answer": "Interest rate risk is the potential for investment losses due to changes in interest rates."
  },
  {
    "prompt": "What is a credit default swap (CDS)?",
    "answer": "A credit default swap is a financial contract that acts like insurance against the default of a borrower or bond issuer."
  },
  {
    "prompt": "What is a balance sheet?",
    "answer": "A balance sheet is a financial statement that shows a company’s assets, liabilities, and shareholders’ equity at a specific point in time."
  },
  {
    "prompt": "What is a P/E ratio?",
    "answer": "The P/E ratio (Price-to-Earnings ratio) measures a company's current share price relative to its per-share earnings, used to value stocks."
  },
  {
    "prompt": "What is compounding in finance?",
    "answer": "Compounding refers to earning interest on both the initial principal and the accumulated interest from previous periods."
  },
  {
    "prompt": "What is a fixed deposit?",
    "answer": "A fixed deposit is a savings account where money is deposited for a fixed period at a predetermined interest rate."
  },
  {
    "prompt": "What is a money market fund?",
    "answer": "A money market fund is a type of mutual fund that invests in short-term, low-risk debt instruments like Treasury bills and commercial paper."
  },
  {
    "prompt": "What is a Treasury bond?",
    "answer": "A Treasury bond is a long-term debt security issued by a government to finance its spending, typically considered very low risk."
  },
  {
    "prompt": "What is quantitative easing?",
    "answer": "Quantitative easing (QE) is a monetary policy where a central bank buys government securities to inject money into the economy and lower interest rates."
  },
  {
    "prompt": "What is leverage?",
    "answer": "Leverage refers to using borrowed funds to increase potential returns on investment, though it also increases risk."
  },
  {
    "prompt": "What is risk appetite?",
    "answer": "Risk appetite is the amount of risk an individual or organization is willing to accept in pursuit of financial goals."
  },
  {
    "prompt": "What is a credit rating?",
    "answer": "A credit rating assesses the creditworthiness of a borrower or debt issuer, indicating the likelihood of default."
  },
  {
    "prompt": "What is a hedge fund?",
    "answer": "A hedge fund is a private investment fund that uses complex strategies, including leverage and derivatives, to maximize returns."
  },
  {
    "prompt": "What is private equity?",
    "answer": "Private equity refers to investments made in private companies that are not listed on public exchanges, often involving direct ownership."
  },
  {
    "prompt": "What is venture capital?",
    "answer": "Venture capital is funding provided to startups and small businesses with high growth potential in exchange for equity."
  },
  {
    "prompt": "What is a current account deficit?",
    "answer": "A current account deficit occurs when a country imports more goods, services, and capital than it exports."
  },
  {
    "prompt": "What is foreign exchange (Forex)?",
    "answer": "Foreign exchange, or Forex, is the global marketplace for buying and selling currencies."
  },
  {
    "prompt": "What is purchasing power parity (PPP)?",
    "answer": "Purchasing power parity is an economic theory that compares different countries’ currencies through a 'basket of goods' approach."
  },
  {
    "prompt": "What is the difference between nominal and real GDP?",
    "answer": "Nominal GDP measures output using current prices, while real GDP adjusts for inflation to reflect true economic growth."
  },
  {
    "prompt": "What is the yield curve?",
    "answer": "The yield curve is a graph that shows the relationship between interest rates and bond maturities, often used to predict economic trends."
  },
  {
    "prompt": "What is monetary inflation?",
    "answer": "Monetary inflation occurs when the money supply grows faster than the economy’s ability to produce goods and services, leading to price increases."
  },
  {
    "prompt": "What is moral hazard in finance?",
    "answer": "Moral hazard occurs when one party takes on risk because they know another party will bear the cost if things go wrong."
  },
  {
    "prompt": "What is a fintech company?",
    "answer": "A fintech company uses technology to improve or automate financial services, such as digital payments or online lending."
  },
  {
    "prompt": "What is a credit union?",
    "answer": "A credit union is a member-owned financial cooperative that provides traditional banking services, often at lower fees and better rates."
  },
  {
    "prompt": "What is an overdraft?",
    "answer": "An overdraft occurs when withdrawals from a bank account exceed the available balance, resulting in a negative balance."
  },
  {
    "prompt": "What is an asset-backed security (ABS)?",
    "answer": "An asset-backed security is a financial product backed by a pool of assets such as loans, leases, or credit card receivables."
  },
  {
    "prompt": "What is the role of the IMF?",
    "answer": "The International Monetary Fund (IMF) promotes global economic stability by providing financial assistance and policy advice to member countries."
  },
  {
    "prompt": "What is the World Bank’s primary goal?",
    "answer": "The World Bank’s main goal is to reduce poverty by providing financial and technical assistance for development projects in low- and middle-income countries."
  },
  {
    "prompt": "What does 'too big to fail' mean?",
    "answer": "‘Too big to fail’ refers to large financial institutions whose failure could cause widespread economic harm, often leading to government support during crises."
  },
  {
    "prompt": "What is an exchange rate?",
    "answer": "An exchange rate is the price of one currency in terms of another, determining how much foreign currency you get for your own."
  },
  {
    "prompt": "What is a trade deficit?",
    "answer": "A trade deficit occurs when a country imports more goods and services than it exports."
  },
  {
    "prompt": "What is asset allocation?",
    "answer": "Asset allocation is the process of dividing investments among different asset classes, such as stocks, bonds, and cash, to balance risk and return."
  },
  {
    "prompt": "What is behavioral finance?",
    "answer": "Behavioral finance studies how psychological factors influence investor behavior and market outcomes."
  },
  {
    "prompt": "What is capital adequacy ratio (CAR)?",
    "answer": "CAR measures a bank’s capital relative to its risk-weighted assets, ensuring it can absorb potential losses."
  },
  {
    "prompt": "What is insider trading?",
    "answer": "Insider trading is buying or selling securities based on non-public, material information about a company."
  },
  {
    "prompt": "What is financial leverage?",
    "answer": "Financial leverage refers to using borrowed funds to amplify returns from an investment, but it also increases potential losses."
  },
  {
    "prompt": "What is sovereign debt?",
    "answer": "Sovereign debt is money borrowed by a national government, often through issuing bonds."
  },
  {
    "prompt": "What is GDP per capita?",
    "answer": "GDP per capita is the total GDP of a country divided by its population, showing the average economic output per person."
  },
  {
    "prompt": "What is a budget deficit?",
    "answer": "A budget deficit occurs when a government’s spending exceeds its revenue in a given period."
  },
  {
    "prompt": "What is a yield?",
    "answer": "Yield is the income return on an investment, such as interest or dividends, expressed as a percentage of the investment’s cost or value."
  },
  {
    "prompt": "What does diversification help reduce?",
    "answer": "Diversification helps reduce unsystematic risk — the risk specific to a single asset or company."
  },
  {
    "prompt": "What is short selling?",
    "answer": "Short selling is a trading strategy where an investor sells borrowed shares, expecting to buy them back later at a lower price to profit from a decline."
  },
  {
    "prompt": "What is a margin call?",
    "answer": "A margin call occurs when the value of an investor’s margin account falls below the required level, prompting the broker to demand more funds."
  },
  {
    "prompt": "What is blockchain?",
    "answer": "Blockchain is a decentralized digital ledger that records transactions securely across multiple computers."
  },
  {
    "prompt": "What is cryptocurrency?",
    "answer": "Cryptocurrency is a digital or virtual currency that uses cryptography for security and operates on decentralized networks like blockchain."
  },
  {
    "prompt": "What is Bitcoin?",
    "answer": "Bitcoin is the first and most well-known cryptocurrency, created in 2009 as a decentralized digital alternative to money."
  },
  {
    "prompt": "What is market volatility?",
    "answer": "Market volatility refers to the degree of variation in trading prices over time, indicating how much and how quickly prices change."
  },
  {
    "prompt": "What is a portfolio?",
    "answer": "A portfolio is a collection of financial assets like stocks, bonds, and cash held by an investor."
  },
  {
    "prompt": "What is a blue-chip stock?",
    "answer": "A blue-chip stock belongs to a large, reputable, and financially stable company with a history of reliable performance."
  },
  {
    "prompt": "What is the efficient market hypothesis (EMH)?",
    "answer": "EMH suggests that asset prices fully reflect all available information, making it hard to consistently outperform the market."
  },
  {
    "prompt": "What is liquidity risk?",
    "answer": "Liquidity risk is the risk of not being able to quickly sell an asset without significantly affecting its price."
  },
  {
    "prompt": "What is a financial bubble?",
    "answer": "A financial bubble occurs when asset prices rise far above their intrinsic value, often followed by a sudden collapse."
  },
  {
    "prompt": "What is the difference between value investing and growth investing?",
    "answer": "Value investing focuses on undervalued stocks, while growth investing targets companies expected to grow faster than the market."
  },
  {
    "prompt": "What is an annuity?",
    "answer": "An annuity is a financial product that provides regular payments to an individual, often used for retirement income."
  },
  {
    "prompt": "What is a credit spread?",
    "answer": "A credit spread is the difference in yield between a corporate bond and a government bond of similar maturity, reflecting credit risk."
  },
  {
    "prompt": "What is an exchange-traded note (ETN)?",
    "answer": "An ETN is a debt security that tracks a market index or benchmark, issued by banks and traded like a stock."
  },
  {
    "prompt": "What is monetary policy tightening?",
    "answer": "Monetary policy tightening involves raising interest rates or reducing money supply to control inflation."
  },
  {
    "prompt": "What is stagflation?",
    "answer": "Stagflation is an economic situation where high inflation and unemployment occur simultaneously with stagnant growth."
  },
  {
    "prompt": "What is the role of credit bureaus?",
    "answer": "Credit bureaus collect and maintain credit information on individuals and businesses, used by lenders to assess creditworthiness."
  },
  {
    "prompt": "What is capital gains tax?",
    "answer": "Capital gains tax is a tax on the profit made from selling an asset for more than its purchase price."
  },
  {
    "prompt": "What is monetary base?",
    "answer": "The monetary base consists of currency in circulation and reserves held by banks at the central bank."
  },
  {
    "prompt": "What is a callable bond?",
    "answer": "A callable bond can be redeemed by the issuer before its maturity date, usually when interest rates fall."
  },
  {
    "prompt": "What is a convertible bond?",
    "answer": "A convertible bond can be converted into a predetermined number of shares of the issuing company."
  },
  {
    "prompt": "What is economic globalization?",
    "answer": "Economic globalization refers to the increasing integration and interdependence of national economies through trade, investment, and capital flows."
  },
  {
    "prompt": "What is microfinance?",
    "answer": "Microfinance provides small loans and financial services to low-income individuals or small businesses without access to traditional banking."
  },
  {
    "prompt": "What is an arbitrage opportunity?",
    "answer": "Arbitrage occurs when an investor exploits price differences for the same asset across different markets to make a risk-free profit."
  },
  {
    "prompt": "What is a debt-to-equity ratio?",
    "answer": "The debt-to-equity ratio measures a company’s financial leverage by comparing its total debt to shareholders’ equity."
  },
  {
    "prompt": "What is compound annual growth rate (CAGR)?",
    "answer": "CAGR measures the mean annual growth rate of an investment over a specified time longer than one year."
  },
  {
    "prompt": "What is a financial intermediary?",
    "answer": "A financial intermediary is an institution, like a bank or mutual fund, that connects savers and borrowers."
  },
  {
    "prompt": "What is a sovereign wealth fund?",
    "answer": "A sovereign wealth fund is a state-owned investment fund that invests in assets like stocks, bonds, or real estate using national reserves."
  },
  {
    "prompt": "What is a payday loan?",
    "answer": "A payday loan is a short-term, high-interest loan typically repaid on the borrower’s next payday."
  }

]


# Convert to pandas DataFrame and then to a Hugging Face Dataset object
df = pd.DataFrame(bfsi_data)
dataset = Dataset.from_pandas(df)

def format_data(example):
    messages = [
        {"role": "system", "content": "You are a specialized BFSI expert who provides concise and accurate financial definitions."},
        {"role": "user", "content": example["prompt"]},
        {"role": "assistant", "content": example["answer"]},
    ]
    # The tokenizer handles the correct formatting of the messages into a single string
    example["text"] = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False
    )
    return example



--- 2. Preparing Synthetic BFSI Dataset ---


In [ ]:



# 4. Model and Tokenizer Setup

print("\n--- 3. Loading Model and Tokenizer (4-bit Quantization) ---")

# 4-bit quantization configuration
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# Load the model with 4-bit configuration
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map="auto",
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
# Set the padding token to be the EOS token for the model
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Important for Causal LMs

# Apply the formatting function to the dataset
processed_dataset = dataset.map(format_data, remove_columns=['prompt', 'answer'])


--- 3. Loading Model and Tokenizer (4-bit Quantization) ---


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Map:   0%|          | 0/92 [00:00<?, ? examples/s]

In [ ]:

# 5. QLoRA and SFT Configuration

print("\n--- 4. Configuring QLoRA and Training Arguments ---")

# PEFT (Parameter-Efficient Fine-Tuning) configuration for LoRA
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",
    task_type="CAUSAL_LM",
    # Target all linear layers in Phi-3
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ]
)

# Training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim="paged_adamw_8bit",
    save_steps=0, # Don't save checkpoints during training
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=0.001,
    fp16=False,
    bf16=True, # Use bfloat16 for better performance on modern GPUs (V100, T4, A100, H100)
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="none", # Disable reporting for Colab simplicity
)

# Supervised Fine-Tuning Trainer
trainer = SFTTrainer(
    model=model,
    train_dataset=processed_dataset,
    peft_config=peft_config,
    args=training_args,
)



--- 4. Configuring QLoRA and Training Arguments ---


Adding EOS to train dataset:   0%|          | 0/92 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/92 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/92 [00:00<?, ? examples/s]

In [ ]:

# 6. Fine-Tuning Execution

print("\n--- 5. Starting Fine-Tuning (Training Phase) ---")
trainer.train()

# Save the fine-tuned LoRA adapters
trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)
print(f"\n✅ Fine-tuning complete. LoRA adapters saved to {output_dir}")


--- 5. Starting Fine-Tuning (Training Phase) ---


Step,Training Loss
10,1.634800
20,0.483200



✅ Fine-tuning complete. LoRA adapters saved to ./phi3-bfsi-finetuned


In [ ]:
# --- NEW MEMORY FIX: Force Cleanup of Training Objects ---
import gc
#del trainer
#del model # Delete the model object from the training phase
torch.cuda.empty_cache() # Clear CUDA cache
gc.collect() # Force garbage collection


# 7. Model Testing and Verification (Inference)
#
# This section loads the base model, applies the saved adapter, and runs
# inference to see the change in the model's response.

print("\n--- 6. Loading Fine-Tuned Model for Inference ---")

# Reload the tokenizer with the correct padding settings
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left" # Set to left for generation

# Define max memory to allow offloading to CPU/RAM during reload
max_memory = {0: "12GB", "cpu": "30GB"} # Adjust these limits based on your environment. 12GB for T4 GPU is a conservative estimate.

# Reload the base model in 4-bit
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    return_dict=True,
    # REUSED bnb_config for 4-bit loading
    quantization_config=bnb_config,
    # Added low_cpu_mem_usage=True to aggressively manage memory during loading
    low_cpu_mem_usage=True,
    # Re-enabled device_map="auto" and added max_memory to explicitly allow CPU offload
    device_map="auto",
    max_memory=max_memory,
)

# Load the PEFT adapter onto the base model
model = PeftModel.from_pretrained(base_model, output_dir)
# Removed model.merge_and_unload() to save memory. We use the PeftModel directly for inference.

# Test Prompt: A question that the model learned during fine-tuning.
test_question = "Define 'systemic risk' in the context of banking."

# Format the prompt using the chat template
messages = [
    {"role": "system", "content": "You are a specialized BFSI expert who provides concise and accurate financial definitions."},
    {"role": "user", "content": test_question},
]
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)


# Generate the response
print(f"\n❓ Test Question:\n{test_question}")

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # Removed device=device to avoid conflict with device_map="auto"
)

generation_args = {
    "max_new_tokens": 100,
    "temperature": 0.1, # Keep it low for factual Q&A
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id
}

result = pipe(prompt, **generation_args)
generated_text = result[0]['generated_text']

# Post-process the output to isolate the assistant's response
try:
    # Find the beginning of the assistant's response in the ChatML format
    # The output will look something like: "<|system|>...<|end|><|user|>...<|end|><|assistant|>..."
    assistant_prefix = "<|assistant|>"
    response_start_index = generated_text.rfind(assistant_prefix)
    if response_start_index != -1:
        # Extract content after the assistant prefix and strip any trailing chat markers
        final_answer = generated_text[response_start_index + len(assistant_prefix):].strip()
        # Clean up the end token if it's there
        final_answer = final_answer.replace("<|end|>", "").strip()
    else:
        final_answer = generated_text
except Exception:
    final_answer = generated_text


print("\n💡 Fine-Tuned Model Response:")
print(final_answer)




--- 6. Loading Fine-Tuned Model for Inference ---


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0



❓ Test Question:
Define 'systemic risk' in the context of banking.

💡 Fine-Tuned Model Response:
Systemic risk refers to the potential collapse of an entire financial system or market, due to the failure of a single entity or a group of entities, which can lead to a cascading failure.


In [ ]:
import shutil
from google.colab import files
import os

# --- Configuration ---
# This is the directory created by your fine-tuning script
ADAPTER_DIRECTORY = "./phi3-bfsi-finetuned"
# This is the name of the zip file that will be downloaded
ZIP_FILENAME = "phi3-bfsi-adapters.zip"

print(f"Starting compression of '{ADAPTER_DIRECTORY}'...")

# 1. Check if the directory exists
if not os.path.isdir(ADAPTER_DIRECTORY):
    print(f"Error: Directory '{ADAPTER_DIRECTORY}' not found. Please ensure your fine-tuning run completed successfully.")
else:
    # 2. Compress the entire directory into a single ZIP file
    # shutil.make_archive(base_name, format, root_dir, base_dir)
    # The output will be {ZIP_FILENAME}
    shutil.make_archive(ZIP_FILENAME.split('.')[0], 'zip', root_dir='.', base_dir=ADAPTER_DIRECTORY)

    print(f"Compression complete: '{ZIP_FILENAME}' created.")

    # 3. Trigger the browser download to your local computer
    print("Initiating file download to your local machine...")
    files.download(f"{ZIP_FILENAME}")

    print("\nDownload initiated. You can now use this ZIP file for self-hosting deployment (Path 1).")


Starting compression of './phi3-bfsi-finetuned'...
Compression complete: 'phi3-bfsi-adapters.zip' created.
Initiating file download to your local machine...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Download initiated. You can now use this ZIP file for self-hosting deployment (Path 1).


In [ ]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 17.3 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig # Added BitsAndBytesConfig
from peft import PeftModel
import os

model = None
tokenizer = None

# --- Configuration ---
# 1. Base Model ID (The model you started fine-tuning with)
BASE_MODEL_ID = "microsoft/Phi-3-mini-4k-instruct"

# 2. Local directory where you unzipped the LoRA adapters

LORA_ADAPTERS_PATH = "./phi3-bfsi-finetuned"

# --- Device Setup ---
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")
if device == "cpu":
    print("Warning: Inference on CPU will be significantly slower.")
    # For CPU, load the model with the standard bfloat16 dtype
    load_kwargs = {"torch_dtype": torch.bfloat16}
else:
    # For GPU (CUDA), use 4-bit quantization to save memory
    print("Applying 4-bit quantization (BitsAndBytes) to conserve GPU memory.")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    load_kwargs = {"quantization_config": bnb_config, "device_map": "auto"}


# --- 1. Load Base Model and Tokenizer ---
print("1. Loading base model and tokenizer...")
try:
    # Load the base model (e.g., Phi-3) using memory-efficient settings for GPU/CPU
    model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_ID,
        **load_kwargs, # Use the defined loading arguments (quantization or dtype)
        trust_remote_code=True
    )
    # Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
except Exception as e:
    print(f"Error loading base model: {e}")
    exit()

# --- 2. Load and Merge LoRA Adapters ---
print(f"2. Loading LoRA adapters from {LORA_ADAPTERS_PATH} and merging...")
try:
    # Load the LoRA adapter weights
    model = PeftModel.from_pretrained(model, LORA_ADAPTERS_PATH)


    model = model.merge_and_unload()
    model.eval()

    print("LoRA adapters successfully merged!")
except Exception as e:
    print(f"Error loading or merging LoRA adapters. Did you unzip the file into '{LORA_ADAPTERS_PATH}'? Error: {e}")
    exit()

# --- 3. Initialize Chat History and Pipeline ---
# Conversation history starts with the system prompt to set the persona
messages = [
    {"role": "system", "content": "You are a specialized financial analyst (BFSI). Answer all queries concisely, clearly, and based on your expert financial knowledge."},
]

# Create the text generation pipeline
generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # The device is now automatically handled by device_map="auto" (for GPU) or runs on CPU
)

# Generation settings
generation_args = {
    "max_new_tokens": 128,
    "temperature": 0.7,
    "do_sample": True,
    "return_full_text": False, # Only return the generated response, not the prompt
    "use_cache": False, # FIX: Disable cache to prevent 'seen_tokens' error with PEFT/quantization
}

print("\n--- PHI-3 BFSI CHATBOT READY ---")
print("Ask me anything about banking, finance, or insurance! (Type 'quit' or 'exit' to end)")
print("---------------------------------\n")

# --- 4. Interactive Chat Loop ---
while True:
    user_input = input("You: ").strip()

    if user_input.lower() in ['quit', 'exit']:
        print("Exiting chat. Goodbye!")
        break

    if not user_input:
        continue

    # Add user message to history
    messages.append({"role": "user", "content": user_input})

    # Apply chat template for the entire conversation history
    # This prepares the prompt in the format the Phi-3 model expects, including history
    formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    try:
        # Generate the response
        output = generator(formatted_prompt, **generation_args)
        # Extract the generated text and clean up whitespace
        generated_text = output[0]['generated_text'].strip()

        # Print the response and update the history with the assistant's reply
        print(f"Phi-3 BFSI: {generated_text}")
        messages.append({"role": "assistant", "content": generated_text})

    except Exception as e:
        print(f"An error occurred during generation: {e}")
        # Remove the last user message to allow retry
        messages.pop()


Using device: cuda
Applying 4-bit quantization (BitsAndBytes) to conserve GPU memory.
1. Loading base model and tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

...

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
Device set to use cuda:0


LoRA adapters successfully merged!

--- PHI-3 BFSI CHATBOT READY ---
Ask me anything about banking, finance, or insurance! (Type 'quit' or 'exit' to end)
---------------------------------

You: hello


Phi-3 BFSI: Hello! How can I assist you with your financial analysis today?
You: Tell me about your self
Phi-3 BFSI: As an AI developed by Microsoft, I don't possess personal attributes or a self in the human sense. However, I am a sophisticated natural language processing system designed to assist users with information and problem-solving.
You: “How much should I save for retirement?
Phi-3 BFSI: The amount you should save for retirement varies based on individual circumstances, but a common financial guideline suggests aiming to save between 10% to 15% of your pre-tax income until retirement. Factors such as your expected lifestyle, the age you plan to retire, expected Social Security benefits, and potential employer retirement plans should also be considered to tailor a more accurate savings goal.


KeyboardInterrupt: Interrupted by user

You: hi


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving adapter_config.json to adapter_config.json
Saving adapter_model.safetensors to adapter_model.safetensors
Saving added_tokens.json to added_tokens.json
Saving chat_template.jinja to chat_template.jinja
Saving README.md to README.md
Saving special_tokens_map.json to special_tokens_map.json
Saving tokenizer.json to tokenizer.json
Saving tokenizer.model to tokenizer.model
Saving tokenizer_config.json to tokenizer_config.json
